In [130]:
import pandas as pd 

import numpy as np

import statistics

import math as m

df=pd.read_csv("C:\\Users\\y20cs142\\Desktop\\data.csv")

df

,x1,x2,y
0,-5,5,11
1,-4,4,11
2,-1,1,8
3,2,-3,2
4,2,-2,5
5,3,-2,5
6,3,-3,4


In [131]:
#finding equation 

y_obs = np.transpose([df['y']]) #should use trans otherwise it appends nrmlly row wise

df['c'] = 1 #adding a 1's column temporarily to get x vector

x_t = np.array( [ df['c'], df['x1'], df['x2'] ] ) #x_transpose - as the np.array appends row wise

x = np.transpose( x_t ) #x - x_transpose transpose

first_part = np.linalg.inv( np.matmul ( np.transpose(x), x) ) #inverse(x_transpose*x)

second_part = np.matmul( x_t, y_obs ) #x_transpose*y

beta = np.matmul(first_part, second_part)

beta

y_fitted = beta[0] + (beta[1] * df['x1']) + (beta[2] * df['x2']) #equation beta0+beta1*x1+beta2*x2

y_fitted = list( y_fitted ) #for further calculations

#y_fitted

In [132]:
#testing goodness of fit using coefficient of determination 

df['y_fitted'] = y_fitted #appending fitted value to df

df['error'] = df['y'] - df['y_fitted'] #finding e - error

sse=sum ( df ['error'] ** 2) 

print("sse: \n",sse)

y_bar = statistics.mean( df['y'] )

sst = sum( (df['y']-y_bar) ** 2)

print("sst: \n",sst)

ssr = sst - sse

print("ssr: \n",ssr)

r_square = ssr / sst

if(r_square > 0.90):
    
    print("the model is good fit and r^2 value is: ",r_square)
    
else:
    
    print("the model is not a good fit and r^2 value is: ",r_square)
df   

sse: 
 1.7142857142857144
sst: 
 73.71428571428571
ssr: 
 72.0
the model is good fit and r^2 value is:  0.9767441860465117


,x1,x2,y,c,y_fitted,error
0,-5,5,11,1,11.571429,-0.571429
1,-4,4,11,1,10.571429,0.428571
2,-1,1,8,1,7.571429,0.428571
3,2,-3,2,1,2.571429,-0.571429
4,2,-2,5,1,4.571429,0.428571
5,3,-2,5,1,5.571429,-0.571429
6,3,-3,4,1,3.571429,0.428571


In [133]:
#goodness of fit using ANOVA model

dof_reg = len(beta) - 1 # dof of regression = k - 1

dof_error = len(df) - len(beta) #dof of regreesion = n - k

msr_square = ssr / dof_reg #mean sum of squares 

mse_square = sse / dof_error # mean sum of squares for error

f = msr_square / mse_square

# creation of anova table 

anova_table = {"source of variation" : ["regression", "error" , "total"] , "sum of squares" : [ssr , sse , sst] , "mean sum of squares" : [msr_square , mse_square , " "] , "variance ratio" : [" ", f , " "]}

anova = pd.DataFrame(anova_table)

print(anova)

import scipy.stats

f_table = scipy.stats.f.ppf( q=1-.05 , dfn=dof_reg, dfd=dof_error) #getting table value of f

#inference

if(f > f_table):
    print("we accept the model")
    
else:
    print("we reject the model")

  source of variation  sum of squares mean sum of squares variance ratio
0          regression       72.000000                36.0               
1               error        1.714286            0.428571           84.0
2               total       73.714286                                   
we accept the model


In [134]:
# test of indudival parameters
 
cjj = []

rows , col = first_part.shape

for i in range(rows):
    
    for j in range(col):
        
        if(i==j):
            
            cjj.append(first_part[i,j]) #coefficient matrix of diagonal elements

stand_error = []

#calculation of standard error coloumn

for i in cjj:
    
    stand_error.append( m.sqrt (mse_square * i ) )
    

t=[]

#calculation of t coloumn

for i in range(len(beta)):
    
    t.append(int(beta[i] / (stand_error[i])))

# making the table

test_table = {"predictor" : ["beta 0", "beta 1" , "beta 2"] , "coefficient" :  list(beta) , "standard error" : stand_error , "t" : t}

test_table = pd.DataFrame(test_table)

print(test_table)

  predictor          coefficient  standard error   t
0    beta 0  [6.571428571428571]        0.247436  26
1    beta 1  [1.000000000000007]        0.464621   2
2    beta 2                [2.0]        0.464621   4


In [136]:
#inference

t_table = scipy.stats.t.ppf(q=1-.05/2,df=dof_error)

print(t_table)

for i in range(len(beta)):
    
    if abs(t[i]) > t_table:
        
        print("we reject beta",i)
        print("its contributing the model")
        
    else:
        
        print("we accept beta",i)
        print("its not contributing the model")

2.7764451051977987
we reject beta 0
its contributing the model
we accept beta 1
its not contributing the model
we reject beta 2
its contributing the model
